# Setup

## Imports

In [ ]:
# Numpy, Scipy, Matplotlib
import scipy.interpolate
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

# Drake imports
import pydrake
from pydrake.all import (DiagramBuilder, RigidTransform, MathematicalProgram, RollPitchYaw,
                        RotationMatrix, Meshcat, MeshcatVisualizerParams, MeshcatVisualizerCpp,
                        InverseKinematics, Solve, SpatialInertia, UnitInertia)

# Imports of other project files
import constants
import config

import plant.simulation
import plant.manipulator as manipulator

import ctrl.aux
import plant.pedestal

import visualization

# Other imports
import time

In [ ]:
# Matplotlib configuring
# USE FOR PAPER
# plt.style.use(['science', 'no-latex'])
# font = {'size'   : 14}
# matplotlib.rc('font', **font)
# default_figsize = (2*3,2*2)
# USE GENERALLY
font = {'size'   : 16}
matplotlib.rc('font', **font)
default_figsize = (16,8)

In [ ]:
matplotlib.rcParams['figure.figsize'] = (16,8)
matplotlib.rcParams['lines.linewidth'] = 5
matplotlib.rcParams["axes.xmargin"] = 0

## Drake initialization

In [ ]:
meshcat = Meshcat()
web_url = meshcat.web_url()

### Constants

In [ ]:
w_L = constants.nominal_sys_consts.w_L
h_L = constants.nominal_sys_consts.h_L
pedestal_x = plant.pedestal.PEDESTAL_X_DIM

In [ ]:
contact_body_name = manipulator.data["contact_body_name"]

### Creat MBP

In [ ]:
builder = DiagramBuilder()

mbp_plant, scene_graph = pydrake.multibody.plant.AddMultibodyPlantSceneGraph(
    builder, time_step=config.DT)
mbp_plant.set_stiction_tolerance(constants.v_stiction)
mbp_plant.set_penetration_allowance(0.001)

### Add bodies

In [ ]:
# Arm
manipulator.data["add_plant_function"](
    mbp_plant,
    constants.nominal_sys_consts.m_M,
    constants.nominal_sys_consts.r,
    constants.nominal_sys_consts.mu,
    scene_graph = scene_graph
)

In [ ]:
# Pedestal
plant.pedestal.AddPedestal(mbp_plant)

In [ ]:
# Paper
paper_instance = mbp_plant.AddModelInstance("paper")
paper_dims = [constants.PLYWOOD_LENGTH, w_L, h_L]
paper_body = mbp_plant.AddRigidBody(
    "paper_body0", paper_instance,
    SpatialInertia(1, p_PScm_E=np.array([0., 0., 0.]),
                   G_SP_E=UnitInertia.SolidBox(*paper_dims))
)

mbp_plant.RegisterCollisionGeometry(
    paper_body, RigidTransform(), pydrake.geometry.Box(*paper_dims), "paper_body0",
    pydrake.multibody.plant.CoulombFriction(1,1)
)
mbp_plant.RegisterVisualGeometry(
    paper_body, RigidTransform(), pydrake.geometry.Box(*paper_dims),
    "paper_body0", [0, 1, 0, 1])
mbp_plant.WeldFrames(
    mbp_plant.world_frame(),
    mbp_plant.GetBodyByName("paper_body0").body_frame(),
    RigidTransform(RotationMatrix(
    ), [0, 0, plant.pedestal.PEDESTAL_Z_DIM+h_L/2])
)

In [ ]:
# False body
empty_inertia = SpatialInertia(0, [0, 0, 0], UnitInertia(0, 0, 0))
false_body_instance = mbp_plant.AddModelInstance("false")
mbp_plant.AddRigidBody("false_body", false_body_instance, empty_inertia)

end_effector_z_rotation = mbp_plant.AddJoint(pydrake.multibody.tree.RevoluteJoint(
    "contact_body_name_z_rotation",
    mbp_plant.GetFrameByName(contact_body_name),
    mbp_plant.GetFrameByName("false_body"),
    [0, 0, 1],
    damping=0
))

In [ ]:
mbp_plant.Finalize()

### Build diagram

In [ ]:
meshcat_params = MeshcatVisualizerParams()
vis = MeshcatVisualizerCpp.AddToBuilder(
    builder,
    scene_graph.get_query_output_port(),
    meshcat,
    meshcat_params)

diagram = builder.Build()
context = diagram.CreateDefaultContext()
ik_context = mbp_plant.GetMyContextFromRoot(context)
vis_context = vis.GetMyContextFromRoot(context)

# Create end effector traj

In [ ]:
end_time = 30

In [ ]:
ts = np.arange(0,end_time,0.1);

In [ ]:
joint_position = [
    0,
    w_L - constants.PEDESTAL_Y_DIM/2,
    plant.pedestal.PEDESTAL_Z_DIM + constants.nominal_sys_consts.h_L/2
]

In [ ]:
desired_radius = w_L/2

In [ ]:
theta_Xs = np.interp(
   ts,
    [0, end_time],
    [0, np.pi],
)
ys = joint_position[1] + np.cos(theta_Xs)*desired_radius
zs = joint_position[2] + np.sin(theta_Xs)*desired_radius + constants.nominal_sys_consts.r

In [ ]:
# Margin between edge of the link and the CoM of the manipulator
x_margin = 3*constants.IN_TO_M

# Get initial values

In [ ]:
# Grab bodies and frames
end_effector_body = mbp_plant.GetBodyByName(manipulator.data["contact_body_name"])
end_effector_frame = end_effector_body.body_frame()
false_body_frame = mbp_plant.GetBodyByName("false_body").body_frame()

In [ ]:
# Get initial position
starting_q = np.array([
    -1.90831538,
    -0.74013834,
    2.29209901,
    -2.58333082,
    4.52633766,
    0.60032527,
    -3.10678351,
    np.pi
])

In [ ]:
# Initialize plant + visualizations
mbp_plant.SetPositions(ik_context, starting_q)
vis.Publish(vis_context)

man_pose = mbp_plant.EvalBodyPoseInWorld(ik_context, mbp_plant.GetBodyByName(contact_body_name))
visualization.AddMeshcatTriad(meshcat, "man_pose", X_PT=man_pose)

false_pose = mbp_plant.EvalBodyPoseInWorld(ik_context, mbp_plant.GetBodyByName("false_body"))
visualization.AddMeshcatTriad(meshcat, "false_pose", X_PT=false_pose)

In [ ]:
# Actually grab starting values
starting_ee_pose = mbp_plant.EvalBodyPoseInWorld(ik_context, end_effector_body)
starting_x = starting_ee_pose.translation()[0]
starting_theta_Z = RollPitchYaw(starting_ee_pose.rotation()).vector()[2]

# Run optimization

## Set up geometries for collision checking

In [ ]:
geometry_pairs = [
    # ("paper_body0", contact_body_name),
    ("paper_body0", "panda_hand"),
    ("paper_body0", "panda_link8"),
    ("paper_body0", "panda_link7"),
    ("paper_body0", "panda_link6"),
    ("paper_body0", "panda_link5"),
    ("paper_body0", "panda_link4"),
    ("paper_body0", "panda_link3"),
    ("pedestal_left_body", contact_body_name),
    ("pedestal_left_body", "panda_hand"),
    ("pedestal_left_body", "panda_link8"),
    ("pedestal_left_body", "panda_link7"),
    ("pedestal_left_body", "panda_link6"),
    ("pedestal_left_body", "panda_link5"),
    ("pedestal_left_body", "panda_link4"),
    ("pedestal_left_body", "panda_link3"),
    ("pedestal_right_body", contact_body_name),
    ("pedestal_right_body", "panda_hand"),
    ("pedestal_right_body", "panda_link8"),
    ("pedestal_right_body", "panda_link7"),
    ("pedestal_right_body", "panda_link6"),
    ("pedestal_right_body", "panda_link5"),
    ("pedestal_right_body", "panda_link4"),
    ("pedestal_right_body", "panda_link3"),
    ("pedestal_right_body", "panda_link3"),
    ("pedestal_bottom_body", "panda_hand"),
    ("pedestal_bottom_body", "panda_link8"),
    ("pedestal_bottom_body", "panda_link7"),
    ("pedestal_bottom_body", "panda_link6"),
    ("pedestal_bottom_body", "panda_link5"),
    ("pedestal_bottom_body", "panda_link4"),
    ("pedestal_bottom_body", "panda_link3"),
]

# TODO: get this from joints?
adjacent_pairs = [
    ("panda_link0", "panda_link1"),
    ("panda_link1", "panda_link2"),
    ("panda_link2", "panda_link3"),
    ("panda_link3", "panda_link4"),
    ("panda_link4", "panda_link5"),
    ("panda_link5", "panda_link6"),
    ("panda_link6", "panda_link7"),
    ("panda_link6", "panda_link8"),
    ("panda_link7", "panda_link8"),
    ("panda_link7", "panda_hand"),
    ("panda_link8", "panda_hand"),
]
panda_links = ["panda_link" + str(i) for i in range(9)]
panda_links += ["panda_hand"]
for link_1 in panda_links:
    for link_2 in panda_links:
        if link_1 == link_2:
            continue
        if (link_1, link_2) in adjacent_pairs:
            continue
        if (link_2, link_1) in adjacent_pairs:
            continue
        if (link_2, link_1) in geometry_pairs:
            continue
        if (link_1, link_2) in geometry_pairs:
            continue
        geometry_pairs.append((link_1, link_2))

## Run IK

In [ ]:
# Output arrays
qs = [starting_q]
successes = [True]

for t, theta_X, y, z in zip(ts, theta_Xs, ys, zs):
    # Create program
    ik = InverseKinematics(mbp_plant, ik_context, with_joint_limits=True)
    
    # Add collision contstraints
    for body_name_i, body_name_j in geometry_pairs:
            geo_list = (
                mbp_plant.GetCollisionGeometriesForBody(
                    mbp_plant.GetBodyByName(body_name_i))[0],
                mbp_plant.GetCollisionGeometriesForBody(
                    mbp_plant.GetBodyByName(body_name_j))[0],
            )
            ik.AddDistanceConstraint(
                distance_lower=1e-3, distance_upper=1e9, geometry_pair=geo_list)
    
    # Add position constraint
    p_WG_lower = np.array([[-1e-2, y-1e-2, z-1e-2]]).T
    p_WG_upper = np.array([[1e-2, y+1e-2, z+1e-2]]).T
    ik.AddPositionConstraint(
        frameA=mbp_plant.world_frame(), frameB=end_effector_frame, p_BQ=np.zeros(3),
        p_AQ_lower=p_WG_lower, p_AQ_upper=p_WG_upper)
    
    # Add orientation constraint
    R_WG = RotationMatrix.MakeXRotation(theta_X)
    ik.AddOrientationConstraint(
        frameAbar=mbp_plant.world_frame(), R_AbarA=R_WG,
        frameBbar=false_body_frame, R_BbarB=RotationMatrix(),
        theta_bound=0.01*np.pi
    )

    # Set seed as previous q
    ik.prog().SetInitialGuess(ik.q(), qs[-1])
    
    # Solve and grab values
    result = Solve(ik.prog())
    if not result.is_success():
        print(result.GetInfeasibleConstraintNames(ik.prog()))
    successes.append(result.is_success())
    qs.append(result.GetSolution(ik.q()))

In [ ]:
print("Success rate: {:.2f}%".format(100*np.count_nonzero(successes)/len(successes)))

# Results

## Visualize in meshcat

In [ ]:
t = 0
dt = 0.1
xs_out = []
ys_out = []
zs_out = []
for q, success in zip(qs, successes):
    if success and True:
        mbp_plant.SetPositions(ik_context, q)
        
        # Evaluate end effector pose
        ee_pose = mbp_plant.EvalBodyPoseInWorld(ik_context, end_effector_body)
        xs_out.append(ee_pose.translation()[0])
        ys_out.append(ee_pose.translation()[1])
        zs_out.append(ee_pose.translation()[2])

        # Visualize
        visualization.AddMeshcatTriad(meshcat, "man_pose", X_PT=ee_pose)
        false_pose = mbp_plant.EvalBodyPoseInWorld(
            ik_context, mbp_plant.GetBodyByName("false_body"))
        visualization.AddMeshcatTriad(meshcat, "false_pose", X_PT=false_pose, opacity=0.3)
        
        # Publish and increment time
        vis.Publish(vis_context)
        time.sleep(dt)
        t += dt

## Plot results

In [ ]:
plt.figure()
plt.plot(ys, zs, label="Desired", color='purple', linestyle='--')
plt.plot(ys_out, zs_out, label="Actual", color='purple', alpha=0.3)
plt.gca().set_aspect("equal")